In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib
import matplotlib.pyplot as plt
import random
%matplotlib inline
from matplotlib.dates import DateFormatter, WeekdayLocator, DayLocator, MONDAY

import mysql.connector
from mysql.connector import errorcode
from ittutils import ittconnection, get_resampled, get_raw_price, get_raw_volume, classification_dataset_from_ts

# Create sql for modifieng signals

In [2]:

# change in the real db the sign of trend
update_1_query = (
    " UPDATE signal_signal SET trend = -trend WHERE `signal` = 'ANN_simple' "
)


# Check staging db before update

In [4]:
db_connection = ittconnection('prodcopy')
query = (" SELECT * FROM signal_signal WHERE `signal` = 'ANN_simple' ")
signals_df = pd.read_sql(query, con=db_connection)
signals_df.tail(6)

,id,created_at,modified_at,UI,subscribers_only,text,source,transaction_currency,signal,trend,...,volume_usdt_change,sent_at,timestamp,rsi_value,counter_currency,resample_period,predicted_ahead_for,probability_down,probability_same,probability_up
47899,379831,2018-06-04 09:11:22.857366,2018-06-04 09:11:22.992928,0,1,,1,DCR,ANN_Simple,-1,...,None,1.528103e+09,1.528103e+09,None,0,60,900.0,0.236317,0.514463,0.249220
47900,379839,2018-06-04 09:11:43.131297,2018-06-04 09:11:43.275413,0,1,,1,BLK,ANN_Simple,1,...,None,1.528104e+09,1.528103e+09,None,0,60,900.0,0.315797,0.410273,0.273929
47901,379840,2018-06-04 09:11:46.303260,2018-06-04 09:11:46.453597,0,1,,1,CVC,ANN_Simple,-1,...,None,1.528104e+09,1.528103e+09,None,0,60,900.0,0.224712,0.483558,0.291730
47902,379841,2018-06-04 09:11:57.988838,2018-06-04 09:11:58.167018,0,1,,1,XRP,ANN_Simple,1,...,None,1.528104e+09,1.528103e+09,None,0,60,900.0,0.318531,0.391965,0.289504
47903,379842,2018-06-04 09:12:00.941717,2018-06-04 09:12:01.084990,0,1,,1,EXCL,ANN_Simple,-1,...,None,1.528104e+09,1.528103e+09,None,0,60,900.0,0.349023,0.288950,0.362027
47904,379845,2018-06-04 09:12:11.525787,2018-06-04 09:12:11.666793,0,1,,1,GNO,ANN_Simple,1,...,None,1.528104e+09,1.528103e+09,None,0,60,900.0,0.302029,0.415630,0.282340


# Update Staging DB

In [5]:
#db_connection = ittconnection('prodcopy')
cursor = db_connection.cursor()
cursor.execute(update_1_query)

db_connection.commit()
cursor.close()
db_connection.close()

## check DB after update

In [6]:
db_connection = ittconnection('prodcopy')
query = (" SELECT * FROM signal_signal WHERE `signal` = 'ANN_simple' ")
signals_df = pd.read_sql(query, con=db_connection)
signals_df.tail(5)


,id,created_at,modified_at,UI,subscribers_only,text,source,transaction_currency,signal,trend,...,volume_usdt_change,sent_at,timestamp,rsi_value,counter_currency,resample_period,predicted_ahead_for,probability_down,probability_same,probability_up
47900,379839,2018-06-04 09:11:43.131297,2018-06-04 09:11:43.275413,0,1,,1,BLK,ANN_Simple,-1,...,None,1.528104e+09,1.528103e+09,None,0,60,900.0,0.315797,0.410273,0.273929
47901,379840,2018-06-04 09:11:46.303260,2018-06-04 09:11:46.453597,0,1,,1,CVC,ANN_Simple,1,...,None,1.528104e+09,1.528103e+09,None,0,60,900.0,0.224712,0.483558,0.291730
47902,379841,2018-06-04 09:11:57.988838,2018-06-04 09:11:58.167018,0,1,,1,XRP,ANN_Simple,-1,...,None,1.528104e+09,1.528103e+09,None,0,60,900.0,0.318531,0.391965,0.289504
47903,379842,2018-06-04 09:12:00.941717,2018-06-04 09:12:01.084990,0,1,,1,EXCL,ANN_Simple,1,...,None,1.528104e+09,1.528103e+09,None,0,60,900.0,0.349023,0.288950,0.362027
47904,379845,2018-06-04 09:12:11.525787,2018-06-04 09:12:11.666793,0,1,,1,GNO,ANN_Simple,-1,...,None,1.528104e+09,1.528103e+09,None,0,60,900.0,0.302029,0.415630,0.282340
